In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd
from html_table_parser import parser_functions
import os
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [8]:
os.chdir('C://Users//nhw85//OneDrive//바탕 화면//[KBO] Batter_Pitcher2vec//DB')

In [3]:
data_files = os.listdir()

total_df = pd.DataFrame()

for file in data_files:
    if not file.endswith('.csv'):
        continue

    abspath = os.path.abspath(file)
    
    temp_df = pd.read_csv(abspath, encoding='utf-8', index_col=0)
    total_df = pd.concat([total_df, temp_df], axis=0)

In [4]:
total_df[total_df['batter_ID']==66643] # 크롤링이 잘못 되었다. 헥터가 pitcher 로 들어가야 하나 batter 자리로 들어갔다.

,gameID,pitcher,batter,pitcher_ID,batter_ID,pitch_result_x,pa_result,pa_result_detail,pitch_result_y,sequence_id,player_id,retro_id,player_name,birth_date,throwing_hand,batting_hand,position
23270,20180606HTKT02018,심재민,헥터,64017.0,66643.0,헛스윙,삼진,삼진,스트라이크 헛스윙 파울 볼 헛스윙,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# # pitcher_ID 리스트 만들기
# p_id = []
# for id in total_df['pitcher_ID']:
#     if id not in p_id:
#         p_id.append(id)

# # batter_ID 리스트 만들기

# b_id = []
# for id in total_df['batter_ID']:
#     if id not in b_id:
#         b_id.append(id)

In [10]:
# total_id = p_id + b_id
# print(len(total_id)) # 956

956


- 956 이라는 수가 정확한 수가 아니라는 것!

In [43]:
# pitcher_name 리스트 만들기

p_name = []
seen_pitcher_ids = set()
pit_value  = {}

for index, row in total_df.iterrows():
    pitcher = row['pitcher']
    pitcher_id = row['pitcher_ID']
    
    if pitcher_id not in seen_pitcher_ids:
        p_name.append(pitcher)
        pit_value[pitcher_id] = pitcher
        seen_pitcher_ids.add(pitcher_id)
        
# # batter_name 리스트 만들기

b_name = []
seen_batter_ids = set()
bat_value  = {}

for index, row in total_df.iterrows():
    batter = row['batter']
    batter_id = row['batter_ID']
    
    if batter_id not in seen_batter_ids:
        b_name.append(batter)
        bat_value[batter_id] = batter
        seen_batter_ids.add(batter_id)

In [65]:
cnt = 0
for i in list(pit_value):
    for j in list(bat_value):
        if i == j:
            print(i)
            cnt += 1
print(cnt) # 잘못 크롤링된 선수들의 수

66643
65062
71118
78148
75340
73211
67646
77263
62929
65643
67640
78536
61962
68701
65618
68556
79130
69439
78517
68050
62908
69737
22


In [69]:
# 956 -> 934 로 줄어든 이유 => 중복된 값을 없앴기 때문!
merged_dict = {**pit_value, **bat_value}

#### Final
playerID_name = merged_dict 

#### 네이버 중계 야구 사이트

- ex) https://m.sports.naver.com/player/index?from=sports&playerId=79764&category=kbo&tab=profile
- 은퇴한 선수들의 경기기록은 존재하나 아이디로 검색을 하였을 때 존재하지 않은 페이지가 많이 나와서 크롤링 중단

#### 스탯티즈로 전환

In [101]:
# total_id를 기준으로 KBO 선수의 기록실 생년월일 수집!

birth_date_list = []
player_id = []
name = []

nonexistent_id = []
nonexistent_birth_date_list = []
nonexistent_name = []

for id in playerID_name:
    url = 'https://m.sports.naver.com/player/index?from=sports&playerId={}&category=kbo&tab=profile'.format(id) 
    # 웹 페이지에 접속하여 HTML 가져오기
    response = requests.get(url)
    response = response.text

    # # BeautifulSoup를 사용하여 HTML 파싱
    soup = BeautifulSoup(response, 'html.parser')
    date_string = soup.find('span' ,{'class':'year'})
    if date_string:
        player_id.append(id)
        birth_date_list.append(date_string.text)
        name.append(playerID_name[id])
    else:
        nonexistent_id.append(id)
        nonexistent_birth_date_list.append(date_string)
        nonexistent_name.append(playerID_name[id])

In [138]:
playerID = player_id + nonexistent_id
result = birth_date_list + nonexistent_birth_date_list
playername = name + nonexistent_name

In [139]:
personal_player_ifo = pd.DataFrame()
personal_player_ifo['player_id'] = playerID
personal_player_ifo['player_name'] = playername
personal_player_ifo['birth_date'] = result

In [140]:
personal_player_ifo.isnull()

,player_id,player_name,birth_date
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
929,False,False,True
930,False,False,True
931,False,False,True
932,False,False,True


In [141]:
os.getcwd()

'C:\\Users\\nhw85\\OneDrive\\바탕 화면\\[KBO] Batter_Pitcher2vec\\DB'

In [161]:
personal_player_ifo['birth_date'] = personal_player_ifo['birth_date'].astype('str') 
copied_sample_df = personal_player_ifo['birth_date'].copy() 
for idx, date in enumerate(copied_sample_df):
    if ',' in date:
        copied_sample_df[idx] = date.split(',')[0]
    elif '음력' in date:
        copied_sample_df[idx] = date.replace('음력', '')

copied_sample_df

personal_player_ifo['birth_date'] = copied_sample_df     
# personal_player_ifo.to_csv('KBO'+'_' + "player_ifo" + '.csv', encoding = 'utf-8')

In [9]:
KBO_personal_player_ifo = pd.read_csv("C:/Users/nhw85/OneDrive/바탕 화면/[KBO] Batter_Pitcher2vec/DB/KBO_personal_player_ifo.csv", encoding = 'utf-8',index_col=0)
KBO_personal_player_ifo

,player_id,player_name,birth_date
0,61240,니퍼트,1981년 5월 6일
1,67742,비야누에바,1983년 11월 28일
2,74731,송창식,1985년 3월 25일
3,99737,박정진,1976년 5월 27일
4,79764,장민재,1990년 3월 19일
...,...,...,...
929,76746,김태완,1984년 01월 27일
930,64724,박준혁,1991년 01월 09일
931,67134,로니,1984년 05월 07일
932,61865,이윤재,1989년 01월 02일


In [10]:
date_list = []

for b_date in KBO_personal_player_ifo['birth_date']:
    date = datetime.strptime(b_date.strip(), '%Y년 %m월 %d일')
    formatted_date = date.strftime('%Y-%m-%d')
    date_list.append(formatted_date)

In [11]:
KBO_personal_player_ifo['birth_date'] = date_list
KBO_personal_player_ifo

,player_id,player_name,birth_date
0,61240,니퍼트,1981-05-06
1,67742,비야누에바,1983-11-28
2,74731,송창식,1985-03-25
3,99737,박정진,1976-05-27
4,79764,장민재,1990-03-19
...,...,...,...
929,76746,김태완,1984-01-27
930,64724,박준혁,1991-01-09
931,67134,로니,1984-05-07
932,61865,이윤재,1989-01-02


스탯티즈 url 형식: http://www.statiz.co.kr/player.php?name=%EC%95%88%EC%9A%B0%EC%A7%84&birth=1999-08-30
- 심수창 선수 생년월일: 1981-07-09 
- 스탯티즈에서는 1981-02-09 로 되어있음!

In [ ]:
import requests
from bs4 import BeautifulSoup

birth_date = []
throwing_hand = []
batting_hand = []
position = []


for idx in range(len(KBO_personal_player_ifo)):
    url = 'http://www.statiz.co.kr/player.php?name={}&birth={}'.format(KBO_personal_player_ifo['player_name'][idx], KBO_personal_player_ifo['birth_date'][idx])

    # # 웹 페이지에 접속하여 HTML 가져오기
    response = requests.get(url)
    response = response.text

    # # # BeautifulSoup를 사용하여 HTML 파싱
    soup = BeautifulSoup(response, 'html.parser')
    
    # # target_tag = soup.find('span', string='생년월일')
    target_tag2 = soup.find('span', string='투타')
    target_tag3 = soup.find('span', string='통산 포지션')

    if target_tag2 and target_tag3:
        # birth_date.append(target_tag.next_sibling.strip()) # 생년월일
        throwing_hand.append(target_tag2.next_sibling.strip()[:2]) # 투
        batting_hand.append(target_tag2.next_sibling.strip()[2:]) # 타
        position.append(target_tag3.next_sibling.strip()) # 포지션
    else:
        print(KBO_personal_player_ifo['player_name'][idx])
        throwing_hand.append('None') # 투
        batting_hand.append('None') # 타
        position.append('None') # 포지션


In [204]:
KBO_personal_player_ifo['throwing_hand'] = throwing_hand
KBO_personal_player_ifo['batting_hand'] = batting_hand
KBO_personal_player_ifo['position'] = position

In [215]:
KBO_personal_player_ifo[KBO_personal_player_ifo['position']=='?'][:5]

,player_id,player_name,birth_date,throwing_hand,batting_hand,position
592,60648,이인행,1991-06-30,우투,좌타,?
641,66710,강상원,1997-05-05,우투,좌타,?
647,66804,노관현,1993-10-18,우투,좌타,?
650,61742,나성용,1988-01-05,우투,우타,?
656,66968,이재율,1993-05-02,좌투,좌타,?


In [206]:
# KBO_personal_player_ifo.to_csv('KBO'+'_' + "player_ifo2" + '.csv', encoding = 'utf-8')

- 스탯티즈와 KBO 기록실 등 데이터가 없기도 하고 다르기도 해서 나무위키나 위키백과에서 데이터 수기로 작성..

In [13]:
personal_info_df = pd.read_csv("C:/Users/nhw85/OneDrive/바탕 화면/[KBO] Batter_Pitcher2vec/DB\KBO_player_ifo2.csv", encoding='utf-8', index_col=0)
personal_info_df

,player_id,player_name,birth_date,throwing_hand,batting_hand,position
0,61240,니퍼트,1981-05-06,우투,우타,투수
1,67742,비야누에바,1983-11-28,우투,우타,투수
2,74731,송창식,1985-03-25,우투,우타,투수
3,99737,박정진,1976-05-27,좌투,좌타,투수
4,79764,장민재,1990-03-19,우투,우타,투수
...,...,...,...,...,...,...
929,76746,김태완,1984-01-27,우투,우타,지명타자
930,64724,박준혁,1991-01-09,우투,좌타,우익수
931,67134,로니,1984-05-07,좌투,좌타,1루수
932,61865,이윤재,1989-01-02,우투,우타,지명타자


#### Master.csv

In [14]:
Master =pd.DataFrame()
retro_id = []
player_id = []

for id in personal_info_df['player_id']:
    player_id.append(id)
    retro_id.append(str(id) + 'abc')
    
#######
Master['player_id'] = player_id
Master['retro_id'] = retro_id

In [220]:
# Master.to_csv('KBO'+'_' + "Master" + '.csv', encoding = 'utf-8')

In [18]:
sample_df = personal_info_df.copy()
sample_df

,player_id,player_name,birth_date,throwing_hand,batting_hand,position
0,61240,니퍼트,1981-05-06,우투,우타,투수
1,67742,비야누에바,1983-11-28,우투,우타,투수
2,74731,송창식,1985-03-25,우투,우타,투수
3,99737,박정진,1976-05-27,좌투,좌타,투수
4,79764,장민재,1990-03-19,우투,우타,투수
...,...,...,...,...,...,...
929,76746,김태완,1984-01-27,우투,우타,지명타자
930,64724,박준혁,1991-01-09,우투,좌타,우익수
931,67134,로니,1984-05-07,좌투,좌타,1루수
932,61865,이윤재,1989-01-02,우투,우타,지명타자


#### Made new_salary_ifo by using Statiz 

In [41]:
import requests
from bs4 import BeautifulSoup

yearID = []
player = []
playerID = []
Salary = []

# URL을 설정합니다 (name과 birth_date는 적절한 값으로 대체해야 합니다)
for idx in range(len(sample_df)):
    name = sample_df['player_name'][idx]
    birth_date = sample_df['birth_date'][idx]
    player_id = sample_df['player_id'][idx]

    url = "http://www.statiz.co.kr/player.php?opt=11&name={}&birth={}".format(name, birth_date)
    # GET 요청을 보냅니다
    response = requests.get(url)

    # 응답을 파싱합니다
    soup = BeautifulSoup(response.text, 'html.parser')

    # 원하는 데이터를 추출합니다 (예시에서는 테이블의 첫 번째 행에 있는 데이터를 추출합니다)
    rows = soup.find_all('tr', class_='oddrow_stz') + soup.find_all('tr', class_='evenrow_stz')
    
    for row in rows:
        # 각 행에서 필요한 데이터를 추출합니다
        columns = row.find_all('td', recursive=False)
        year = columns[0].text.strip()
        salary = columns[1].text.strip()

    
        if year in ['2017', '2018', '2019', '2020']:
            # print(year, name, player_id, salary)
            yearID.append(year)
            player.append(name)
            playerID.append(player_id)
            Salary.append(salary)

In [95]:
new_salary_ifo = pd.DataFrame()

new_salary_ifo['yearID'] = yearID
new_salary_ifo['player'] = player
new_salary_ifo['playerID'] = playerID
new_salary_ifo['salary'] = Salary

In [122]:
new_salary_ifo = new_salary_ifo.sort_values(by= ['yearID','player'], ascending=[True, True])
new_salary_ifo = new_salary_ifo.reset_index().drop(columns=['index'], axis = 1)

new_salary_ifo["salary"] = new_salary_ifo["salary"].str.replace(',','')
new_salary_ifo

,yearID,player,playerID,salary
0,2017,강경학,61700,6200
1,2017,강구성,62917,2700
2,2017,강동호,67591,2700
3,2017,강민호,74540,100000
4,2017,강상원,66710,2700
...,...,...,...,...
2085,2020,황동재,50441,2700
2086,2020,황영국,64760,2900
2087,2020,황윤호,62908,5000
2088,2020,황인준,65618,3700


In [126]:
new_salary_ifo[new_salary_ifo["salary"] == '']

,yearID,player,playerID,salary
47,2017,김민수,67504,
52,2017,김민혁,65269,
61,2017,김성민,67828,
120,2017,김홍빈,66309,
129,2017,남재현,66660,
...,...,...,...,...
2053,2020,플렉센,50234,
2054,2020,핀토,50835,
2059,2020,한석현,64101,
2068,2020,허관회,69706,


-  스탯티즈에 연봉정보가 없는 선수들은 KBO 공식 기록 사이트, 구글링, 네이버 검색, http://www.kbreport.com/ 등에서 게시된 연봉 정보를 바탕으로 수기로 기입 / 207명

In [123]:
# new_salary_ifo.to_csv('new'+'_' + 'salary_ifo' + '.csv', encoding='utf-8')

In [151]:
new_salary_ifo_1 = pd.read_csv("C:/Users/nhw85/OneDrive/바탕 화면/[KBO] Batter_Pitcher2vec/DB/new_salary_ifo.csv", encoding='utf-8', index_col = 0)
new_salary_ifo_1.isnull().sum()


yearID      0
player      0
playerID    0
salary      0
dtype: int64

In [152]:
new_salary_ifo_1.head()

,yearID,player,playerID,salary
0,2017,강경학,61700,6200
1,2017,강구성,62917,2700
2,2017,강동호,67591,2700
3,2017,강민호,74540,100000
4,2017,강상원,66710,2700
